In [1]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import os
import re
from io import StringIO
import PyPDF2
import pandas as pd
from datetime import datetime 
import datetime as dt
from timeit import default_timer as timer
pd.set_option('display.max_rows', 500)

### Nov

In [2]:
folder = "../Nov2020"
resultFolder = "../Nov2020_Result"
file = "16.11 3207971832_2020-November_eStatement.pdf"
path = folder + '/' + file
excelFile = resultFolder + '/' + re.sub('\.', '_', file.split('-')[0]) + '.xlsx'
excelFile

'../Nov2020_Result/16_11 3207971832_2020.xlsx'

In [3]:
date = re.sub('\.', '/', file.split(' ')[0])
previousDate = (datetime.strptime(date, "%d/%m") - dt.timedelta(days=1)).strftime('%d/%m')
#for nov
date = datetime.strptime(date, "%d/%m").strftime('%d/%m')
date, previousDate

('16/11', '15/11')

In [4]:
def convertPdfToTxt(path):
    startTime = timer()
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    #codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages = maxpages,caching = caching, check_extractable = True):
        interpreter.process_page(page)
    text = retstr.getvalue()
    fp.close()
    device.close()
    retstr.close()
    endTime = timer()
    print("Total time: %0.4fs" % (endTime - startTime))
    return text

text = convertPdfToTxt(path)

#extract no pages
noPage = len(re.findall("NIAGADEBITKREDITBAKIDATETRANSACTIONDEBITCREDITBALANCE", text))

noPage

Total time: 34.4989s


1127

In [5]:
accountInfoDict = {}
accountCol = ["Nombor Akaun / Account Number", 'Jenis Akaun / Account Type',
             'Tarikh Penyata / Statement Date', 'RINGKASAN / SUMMARY']
for no, desc in enumerate(accountCol):
    if no != len(accountCol)-1:
        descIndex = text.index(desc)
        descLen = len(desc)
        nextDescIndex = text.index(accountCol[no + 1])
        result = text[descIndex + descLen: nextDescIndex]
        accountInfoDict[desc] = result
accountInfoDict

{'Nombor Akaun / Account Number': '3207971832',
 'Jenis Akaun / Account Type': 'RM Plus Current Account',
 'Tarikh Penyata / Statement Date': '16 Nov 2020'}

In [6]:
def extractText(desc, startStr, endStr):
    try:
        startStrIndex = desc.index(startStr)
        startStrLen = len(startStr)
        endStrIndex = desc.index(endStr)         
        if startStr == '':
            result = desc[:endStrIndex]
            return result
        elif endStr!='':
            result = desc[startStrIndex + startStrLen:endStrIndex]
            remainDesc = desc[endStrIndex:]
            return result, remainDesc
        else:
            result = desc[startStrIndex + startStrLen:]
            return result
        
    except Exception as e:
#         print(e)
        print("Failed to extract text")

def extractAmount(desc, greedy = True):
#     greedyPattern = '([\d\.\,]+)'
    greedyPattern = '([\d\.\,]+\.\d{2})'
    notGreedyPattern = '([\d\.\,]+?\.\d{2}?)'
    try:
        if greedy:
            result = re.search(greedyPattern, desc).group(1)
            return result
        else:
            result = re.search(notGreedyPattern, desc).group(1)
            return result
    except Exception as e:
#         print(e)
        print("Wrong number pattern")

In [7]:
summaryDict = {}
summaryCol = ['Baki Penutup / Closing Balance',
              'Jumlah Debit / Total Debits', 'Bil. Debit / No. of Debits',
              'Jumlah Kredit / Total Credits', 'Bil. Kredit / No. of Credits']
#extract series of number 
numberSeries = extractText(text, summaryCol[4], 'Muka Surat 1')[0]

#extract closing balance amount
closingBalanceAmount = extractAmount(numberSeries, greedy = False)
summaryDict[summaryCol[0]] = closingBalanceAmount
numberSeries = re.sub(closingBalanceAmount, '', numberSeries)

#extract debit amount
debitAmount = extractAmount(numberSeries, greedy = False)
summaryDict[summaryCol[1]] = debitAmount
numberSeries = re.sub(debitAmount, '', numberSeries)

#credit no credit
noCredit = re.sub(extractAmount(numberSeries, greedy = False), '', numberSeries)
summaryDict[summaryCol[4]] = noCredit
numberSeries = re.sub(noCredit + '$', '', numberSeries)

tempNumberSeries = numberSeries
creditAmount1 = re.search('(\d{2}\,(\d+\,)*\d+\.\d{2})', numberSeries).group(1)
noDebit1 = re.sub(creditAmount1, '', tempNumberSeries)
creditAmount2 = re.search('(\d{1}\,(\d+\,)*\d+\.\d{2})', numberSeries).group(1)
noDebit2 = re.sub(creditAmount2, '', tempNumberSeries)

noDebitList = [noDebit1, noDebit2]
creditAmountList = [creditAmount1, creditAmount2]
noDebitList, creditAmountList

(['12718', '127181'], ['11,355,618.74', '1,355,618.74'])

In [8]:
def returnTableList(no):
    tempList = []
    textCopy = text
    for i in range(1, no + 1):
#     for i in range(1, 3):
        table, remain = extractText(textCopy, 'Page %s of %s'%(i, no), 'Penyata ini dicetak melalui komputer.')
        index = remain.index('Penyata ini dicetak melalui komputer.')
        termLen = len('Penyata ini dicetak melalui komputer.')
        textCopy = remain[index + termLen:]
        table = re.sub("TARIKHURUS NIAGADEBITKREDITBAKIDATETRANSACTIONDEBITCREDITBALANCE", '', table)
        tempList.append(table)
    return tempList

tableList = returnTableList(noPage)

len(tableList)

1127

In [37]:
#extract first line of transaction

#trx date, cheque no., ref1, ref2, ref3, ref4, credit amount, debit amount, balance
resultList = []

#extract content of first table from text
table = tableList[0]

#extract balance amount
balanceAmount, tempText = extractText(table, "Balance From Last Statement", date)

#store record of balance from last statement
# resultList.append([previousDate, '','', '','', "Balance From Last Statement", '', '', balanceAmount])
#trx date, ref1, credit amount, debit amount, balance
resultList.append([previousDate, "Balance From Last Statement", '', '', balanceAmount])

In [38]:
def extractText(desc, startStr, endStr):
    try:
        startStrIndex = desc.index(startStr)
        startStrLen = len(startStr)
        endStrIndex = desc.index(endStr)         
        if startStr == '':
            result = desc[:endStrIndex]
            return result
        elif endStr!='':
            result = desc[startStrIndex + startStrLen:endStrIndex]
            remainDesc = desc[endStrIndex:]
            return result, remainDesc
        else:
            result = desc[startStrIndex + startStrLen:]
            return result
        
    except Exception as e:
#         print(e)
        print("Failed to extract text")

In [39]:
#extract amountList and descList
#steps: 
#1. extract first and second number, store first number in descList
#2. extract desc between first and second number
#3. store desc in descList
#repeat 1 till 3 until closing balance's line

amountList = []
descList = []
closingLine = 0

# for no, table in enumerate([tableList[0]]):
for no, table in enumerate(tableList):
    if no == 0:
        tempText = tempText
    else:
        tempText = table
    # for i in range(3):
    while closingLine == 0:
        #remove date first
        if re.search(date, tempText):
            tempText = re.sub(date, '', tempText)
        #remove 9.10726135944E11: 778
        if no in [778, 1025]:
            tempText = re.sub("9.10726135944E11", '', tempText)
        #extract amount first
        tempAmount1 = extractAmount(tempText)
        #store amount
        amountList.append(tempAmount1)
        #temporary subbed text to get second amount
        tempSubText = re.sub(tempAmount1, '', tempText)
        tempAmount2 = extractAmount(tempSubText)
        tempDesc = extractText(tempSubText, '', tempAmount2)
        
        #if Balance C/F, last line of transaction for that page
        if re.search(r"Balance C/F", str(tempDesc)):
            tempDesc = re.sub(r"Balance C/F", "", str(tempDesc))
            amountList.append(tempAmount2)
            descList.append(tempDesc)
            descList.append("Balance C/F")
            break
            
        #if Balance B/F, first line of transaction for that page
        elif re.search(r"Balance B/F", str(tempDesc)):
            tempDesc = re.sub(r"Balance B/F", "", str(tempDesc))
            descList.append("Balance B/F")
            tempAmount1 = extractAmount(re.sub(date, '', tempText), greedy = False)
            amountList[-1] = tempAmount1
            tempAmount1Index = tempText.index(tempAmount1)
            tempAmount1Len = len(tempAmount1)
            tempText = tempText[tempAmount1Index+tempAmount1Len:]
            
        #if last line for whole file
        elif re.search("Closing Balance In This Statement", str(tempDesc)):
            closingAmount = tempAmount2
            closingAmountIndex = tempSubText.index(closingAmount)
            lastDesc = tempSubText[:closingAmountIndex]
            #sub closing amount with '' in lastDesc
            lastDesc = re.sub("Closing Balance In This Statement", '', lastDesc)
            descList.append(lastDesc)

            #store last line - amount and desc
            amountList.append(closingAmount)
            descList.append("Closing Balance In This Statement")
#             closingLine = 1
            break
        #extracted all ady
        elif not tempAmount2:
#             closingLine = 1
            break
        else:
            tempText = re.sub('^' + re.escape(tempDesc), '', tempSubText)
            descList.append(tempDesc)

In [22]:
tableList[778]

'16/11Balance B/F23,458,445.203207971832PM16112098 EU REFUNDEU REFUND 9112020 AGNES LEE SHAO L1,599.0023,456,846.20DR-ECP 0061873207971832PM16112098 EU REFUNDEU REFUND 9112020 ONG CHONG DOENG1,525.0023,455,321.20DR-ECP 0061893207971832PM16112098 EU REFUNDEU REFUND 9112020 LAI MUI NYUK1,499.0023,453,822.20DR-ECP 0061913207971832PM16112098 EU REFUNDEU REFUND 9112020 LOW HAN BOON2,348.0023,451,474.20DR-ECP 0061933207971832PM16112098 EU REFUNDEU REFUND 9112020 LIM LI YING299.0023,451,175.20DR-ECP 0061953207971832PM16112098 EU REFUNDEU REFUND 9112020 YONG KIA KIAT4,299.0023,446,876.20DR-ECP 0061973207971832PM16112098 EU REFUNDEU REFUND 9112020 9.10726135944E111,500.0023,445,376.20DR-ECP 0061993207971832PM16112098 EU REFUNDEU REFUND 9112020 LIU PEI LIN1,174.0023,444,202.20DR-ECP 0062013207971832PM16112098 EU REFUNDEU REFUND 9112020 CHEAH SUN LING210.0023,443,992.20DR-ECP 0062033207971832PM16112098 EU REFUNDEU REFUND 9112020 TING HOT KIING500.0023,443,492.20DR-ECP 0062053207971832PM16112098 E

In [23]:
tableList[1025]

'16/11Balance B/F23,194,896.60EU REFUND EU REFUND 9112020YONG KIA KIAT0.1023,194,896.50HANDLING CHRG 008891FEE 3207971832PM16112098EU REFUND EU REFUND 91120209.10726135944E110.1023,194,896.40HANDLING CHRG 008892FEE 3207971832PM16112098EU REFUND EU REFUND 9112020LIU PEI LIN0.1023,194,896.30HANDLING CHRG 008893FEE 3207971832PM16112098EU REFUND EU REFUND 9112020CHEAH SUN LING0.1023,194,896.20HANDLING CHRG 008894FEE 3207971832PM16112098EU REFUND EU REFUND 9112020TING HOT KIING0.1023,194,896.10HANDLING CHRG 008895FEE 3207971832PM16112098EU REFUND EU REFUND 9112020LOW CING VOON0.1023,194,896.00HANDLING CHRG 008896FEE 3207971832PM16112098EU REFUND EU REFUND 9112020AILIN WONG0.1023,194,895.90HANDLING CHRG 008897FEE 3207971832PM16112098EU REFUND EU REFUND 9112020JOYCE CHEN YONG0.1023,194,895.80HANDLING CHRG 008898FEE 3207971832PM16112098EU REFUND EU REFUND 9112020LAI BOON TEIK0.1023,194,895.70HANDLING CHRG 008899FEE 3207971832PM16112098EU REFUND EU REFUND 9112020LEE SAN YING0.1023,194,895.60HAN

In [40]:
def splitAmount(desc):
    try:
        #remove first . before num if exists
        desc = re.sub('^\.', '', desc)
        #remove date pattern
        desc = re.sub('^\d{1,2}\.\d{1,2}\.\d{4}', '', desc)
        amountPattern = '(.*?\.\d{2}?)'
        result = re.search(amountPattern, desc).group(1)
        desc2 = re.sub('^' + result, '', desc)
        return result, desc2
    except Exception as e:
        print(e)
        print("Wrong amount pattern")

splitAmountList = [splitAmount(i) for i in amountList]

In [41]:
def checkCreditDebit(amountTuple, previousBalance):
    #exception:
    amount1 = float(re.sub(',', '', amountTuple[0]))
#     currentBalance = re.sub('^\.', '', amountTuple[1])
    currentBalance = amountTuple[1]
#     print(currentBalance)
    if currentBalance !='':
        currentBalance = float(re.sub(',', '', currentBalance))
        previousBalance = float(re.sub(',', '', previousBalance))
        if currentBalance - previousBalance == amount1:
            amountTuple = ('', round(amount1, 2), currentBalance)
        elif previousBalance - currentBalance == amount1:
            amountTuple = (round(amount1, 2), '', currentBalance)
        #if amount 1 is wrong
        elif currentBalance > previousBalance:
            credit  = currentBalance - previousBalance
            amountTuple = ('', round(credit,2), currentBalance)
        elif previousBalance > currentBalance:
            debit  = previousBalance - currentBalance
            amountTuple = (round(debit,2), '', currentBalance)            
    else:
        amountTuple = ('', '', amount1)
    return amountTuple, str(amountTuple[2])

#categorize one amount as credit/debit
splitAmountList2 = []
for no, i in enumerate(splitAmountList):
    if no == 0:
        resultTuple, previousBalanceAmount = checkCreditDebit(i, balanceAmount)
        splitAmountList2.append(resultTuple)
    else:
        resultTuple, previousBalanceAmount = checkCreditDebit(i, previousBalanceAmount)
        splitAmountList2.append(resultTuple)


In [62]:
splitAmountList[723]

('104365.02', ',912.0039,737,571.73')

In [63]:
descList[723]

'MISC DR 0002013207971832PM11112098 COP REFUNDCOP REFUND 6112020 LAU SIEW PING'

In [64]:
tableList[52]

'11/11Balance B/F39,741,215.240.1039,741,215.14HANDLING CHRG 000200FEE 3207971832PM11112001COP REFUND COP REFUND 8112020MOHD ASHRAF BIN0.1039,741,215.04HANDLING CHRG 000201FEE 3207971832PM11112001COP REFUND COP REFUND 8112020MUHAMMAD HASAN B15.1139,741,199.93MISC DR 0002013207971832PM11112003 MERCHANT REDEEMMERCHANT REDEEM 6112020 MAH POOI SAN115.2039,741,084.73DR-ECP 0002013207971832PM11112002 MERCHANT REDEEMMERCHANT REDEEM 6112020 LING TAI NIK601.0039,740,483.73MISC DR 0002013207971832PM11112099 EU REFUNDEU REFUND 4112020 104365.02,912.0039,737,571.73MISC DR 0002013207971832PM11112098 COP REFUNDCOP REFUND 6112020 LAU SIEW PING3,732.0039,733,839.73MISC DR 0002013207971832PM11112004 MERCHANT REDEEMMERCHANT REDEEM 7112020 FONG YIN LEN0.1039,733,839.63HANDLING CHRG 000202FEE 3207971832PM11112001COP REFUND COP REFUND 8112020MOHD ASHRAF BIN0.1039,733,839.53HANDLING CHRG 000203FEE 3207971832PM11112001COP REFUND COP REFUND 8112020TEY CHIN TIANN42.0339,733,797.50DR-ECP 0002033207971832PM11112

In [42]:
colList = ['Trn. Date','Reference 1',
           'Debit Amount', 'Credit Amount', 'Balance']
for no, finalStrList in enumerate(descList):
    tempList =  [date] + [finalStrList]
    tempList +=  [i for i in splitAmountList2[no]]
    resultList.append(tempList)

df = pd.DataFrame(resultList, columns = colList)
df

,Trn. Date,Reference 1,Debit Amount,Credit Amount,Balance
0,15/11,Balance From Last Statement,,,"21,105,274.02"
1,16/11,SI CR FROM 3999XXXXXX,,1.13007e+07,3.24059e+07
2,16/11,RTN ITEM 093116IBG RTN ITEM 20201116 LTCS13201...,,1400,3.24073e+07
3,16/11,RTN ITEM 093185IBG RTN ITEM 20201116 LTCS13201...,,1600,3.24089e+07
4,16/11,RTN ITEM 093186IBG RTN ITEM 20201116 LTCS13201...,,110.74,3.24091e+07
...,...,...,...,...,...
14979,16/11,INSTANT TRSF DR 076986 ONG ENG TUANEU REFUND 0...,999,,2.26198e+07
14980,16/11,INSTANT TRSF DR 077011NANCY KHO TIEE WANG WENG...,727,,2.2619e+07
14981,16/11,INSTANT TRSF DR 077018 NG HSIU LEEEU REFUND 09...,950,,2.26181e+07
14982,16/11,INSTANT TRSF DR 077050 VUN FUNG YINEU REFUND 0...,200,,2.26179e+07


In [24]:
max([i for i in df['Credit Amount'] if type(i)!=str])

23445367.1

In [25]:
df.query("`Credit Amount`==23445367.1")

,Trn. Date,Reference 1,Debit Amount,Credit Amount,Balance
10691,16/11,DR-ECP 0061993207971832PM16112098 EU REFUNDEU ...,,2.34454e+07,2.34454e+07


In [26]:
df.query("`Credit Amount`==23445367.1")['Reference 1'].values

array(['DR-ECP 0061993207971832PM16112098 EU REFUNDEU REFUND 9112020 LIU PEI LIN'],
      dtype=object)

In [43]:
df.loc[df.shape[0]-1]['Balance']

22617874.03

In [44]:
sum([i for i in df['Debit Amount'] if i!=''])

9843018.729997803

In [45]:
trueNoDebit = df.query("`Debit Amount`!=''").shape[0]
trueNoDebit

12718

In [46]:
sum([i for i in df['Credit Amount'] if i!=''])

11355618.74

In [47]:
#confirm no debit
noDebitIndex = noDebitList.index(str(trueNoDebit))
creditAmount = creditAmountList[noDebitIndex]
summaryDict[summaryCol[2]] = trueNoDebit
summaryDict[summaryCol[3]] = creditAmount

summaryDict

{'Baki Penutup / Closing Balance': '22,617,874.03',
 'Jumlah Debit / Total Debits': '9,843,018.73',
 'Bil. Kredit / No. of Credits': '12',
 'Bil. Debit / No. of Debits': 12718,
 'Jumlah Kredit / Total Credits': '11,355,618.74'}

In [49]:
import xlsxwriter 
  

workbook = xlsxwriter.Workbook(excelFile)
worksheet = workbook.add_worksheet() 

# Rows and columns are zero indexed. 
row = 0

for col in accountInfoDict : 
  
    # write operation perform 
    worksheet.write(row, 0, col + ':')
    worksheet.write(row, 1, accountInfoDict[col])
  
    # incrementing the value of row by one 
    # with each iteratons. 
    row += 1
    
for col in summaryDict : 
  
    # write operation perform 
    worksheet.write(row, 0, col + ':')
    worksheet.write(row, 1, summaryDict[col])
  
    # incrementing the value of row by one 
    # with each iteratons. 
    row += 1
    
workbook.close()

from openpyxl import load_workbook
#write pandas 
writer = pd.ExcelWriter(excelFile, engine='openpyxl')
writer.book = load_workbook(excelFile)
# copy existing sheets
writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
reader = pd.read_excel(excelFile)
df.to_excel(writer, sheet_name='Sheet1', startrow = len(reader) + 2, header = True, index = False)
writer.save()